# spectre Benchmarks

In [1]:
import pandas as pd
print(pd.__version__)
start, end = pd.Timestamp('2013-01-02', tz='UTC'), pd.Timestamp('2018-01-02', tz='UTC')

import sys
sys.path.append('..\\..\\spectre')

from spectre import factors, parallel
import pandas as pd
# download data first: https://www.quandl.com/api/v3/datatables/WIKI/PRICES.csv?qopts.export=true&api_key=[yourapi_key]
%time loader = factors.QuandlLoader('../../historical_data/us/prices/quandl/WIKI_PRICES.zip')

0.25.2
Wall time: 6.48 s


In [2]:
engine = factors.FactorEngine(loader)
engine.to_cuda()

In [11]:
engine.remove_all_factors()
engine.add(factors.MA(100), 'ma')
%time dfsp=engine.run(start, end)
dfsp.head(5).append(dfsp.tail(5))

Wall time: 467 ms


ma
date                      asset           
2013-01-02 00:00:00+00:00 A      38.110535
                          AAL    11.776899
                          AAMC         NaN
                          AAN    28.779516
                          AAON   19.606133
2018-01-02 00:00:00+00:00 ZIXI    4.827700
                          ZNGA    3.831000
                          ZOES   13.561499
                          ZTS    66.546745
                          ZUMZ   17.383249

In [4]:
%timeit engine.run(start, end)
%timeit -n 3 -r 10 engine.run(start, end)

469 ms ± 77.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
478 ms ± 56.4 ms per loop (mean ± std. dev. of 10 runs, 3 loops each)


In [5]:
engine.remove_all_factors()
engine.add(factors.EMA(50, adjust=True), 'ema50')
%time dfsp=engine.run(start, end)
dfsp.head(5).append(dfsp.tail(5))

Wall time: 494 ms


ema50
date                      asset           
2013-01-02 00:00:00+00:00 A      38.895058
                          AAL    12.636659
                          AAMC         NaN
                          AAN    28.475159
                          AAON   20.192312
2018-01-02 00:00:00+00:00 ZIXI    4.584949
                          ZNGA    3.923238
                          ZOES   15.168861
                          ZTS    70.057404
                          ZUMZ   19.615654

In [6]:
%timeit engine.run(start, end)
%timeit -n 3 -r 10 engine.run(start, end)

459 ms ± 44.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
400 ms ± 22.3 ms per loop (mean ± std. dev. of 10 runs, 3 loops each)


In [12]:
engine.remove_all_factors()
engine.add(factors.MACD(), 'MACD')
%time dfsp=engine.run(start, end)
dfsp.head(5).append(dfsp.tail(5))

Wall time: 485 ms


MACD
date                      asset           
2013-01-02 00:00:00+00:00 A      16.290844
                          AAL   -13.419068
                          AAMC         NaN
                          AAN     1.425429
                          AAON   -8.229778
2018-01-02 00:00:00+00:00 ZIXI  -24.083271
                          ZNGA  -22.715288
                          ZOES  -18.841688
                          ZTS    45.559116
                          ZUMZ   -7.300494

In [13]:
%timeit engine.run(start, end)
%timeit -n 3 -r 10 engine.run(start, end)

472 ms ± 55.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
511 ms ± 40.3 ms per loop (mean ± std. dev. of 10 runs, 3 loops each)


In [3]:
engine.remove_all_factors()
f = factors.MACD()+factors.RSI()+factors.STOCHF()
engine.add(f.rank().zscore(), 'f')
%time dfsp=engine.run(start, end)
dfsp.head(5).append(dfsp.tail(5))

Wall time: 758 ms


f
date                      asset          
2013-01-02 00:00:00+00:00 A      0.179157
                          AAL    1.000241
                          AAMC        NaN
                          AAN    0.139074
                          AAON  -0.496173
2018-01-02 00:00:00+00:00 ZIXI   0.104178
                          ZNGA  -1.141708
                          ZOES  -0.710821
                          ZTS   -1.021230
                          ZUMZ  -1.103438

In [4]:
%timeit engine.run(start, end)
%timeit -n 3 -r 10 engine.run(start, end)

658 ms ± 17.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
675 ms ± 13 ms per loop (mean ± std. dev. of 10 runs, 3 loops each)


In [9]:
import torch
torch.cuda.max_memory_allocated()/1024**3, 'GB'

(1.2389349937438965, 'GB')